In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [3]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime

In [93]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'

webanno_df, _ = read_webanno(list(Path(tsv_folder).glob('*.tsv')))

CPU times: user 11.5 s, sys: 480 ms, total: 12 s
Wall time: 17.9 s


In [94]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49821/49821 [13:26<00:00, 61.78it/s]


CPU times: user 13min 35s, sys: 8.43 s, total: 13min 43s
Wall time: 13min 42s


In [101]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()

In [102]:
len(ellipses)

7418

In [103]:
%%time
results = resolve_ellipses(ellipses, iob_df)

/mnt/nfs/home/Florian.Borchert/workspace/ggponc_annotation/notebooks/../util/convert_annotations.py:166: PerformanceWarning: indexing past lexsort depth may impact performance.
  sentence = ellipses.loc[idx]
/mnt/nfs/home/Florian.Borchert/workspace/ggponc_annotation/notebooks/../util/convert_annotations.py:168: PerformanceWarning: indexing past lexsort depth may impact performance.
  full_sentence = iob_df.loc[idx]


CPU times: user 1min 19s, sys: 1.02 s, total: 1min 20s
Wall time: 1min 20s


In [104]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4626, 78090, 0.05923933922397234)

In [105]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5202
Forward ellipses 1945 831
Backward ellipses 2955 2718
Complex ellipses 263 180


In [106]:
import random

def show_random(df):
    i = random.randint(0, len(df))
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    for span, res in zip(item.full_span, item.resolution):
        if span == res:
            marker.append(' ' * len(span))
        else:
            marker.append('^' * len(res))
        start_index = res.find(span)
        spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
        solutions.append(res)
    print(' '.join(spans))
    print(' '.join(solutions))
    print(' '.join(marker))
    print(item.file, item.sentence_id, 'fragment:', item.fragment)
    print('...........')
    print('')

In [107]:
for _ in range(0, 10):
    show_random(results[results.fragment])

R1           - bzw . R2 - Situation
R1-Situation  bzw . R2 - Situation
^^^^^^^^^^^^                      
07_malignes-melanom_0027.tsv 13 fragment: True
...........

WHO                - und der TNM-Klassifikation der UICC
WHO-Klassifikation  und der TNM-Klassifikation der UICC
^^^^^^^^^^^^^^^^^^                                     
13_oesophaguskarzinom_0061.tsv 1 fragment: True
...........

Grad 3 /      4 Toxizitäten
Grad 3 / Grad 4 Toxizitäten
         ^^^^^^            
20_harnblasenkarzinom_0283.tsv 29 fragment: True
...........

klinische T1c           - oder T2a-Kategorie
klinische T1c-Kategorie  oder T2a-Kategorie
          ^^^^^^^^^^^^^                    
22_prostatakarzinom_0232.tsv 6 fragment: True
...........

Anpassungs          - und depressive Störungen
Anpassungsstörungen  und depressive Störungen
^^^^^^^^^^^^^^^^^^^                          
21_hodentumoren_0312.tsv 19 fragment: True
...........

Pouch                - vs . Roux-Y-Rekonstruktion
Pouch-Rekonstruktion 

In [109]:
for _ in range(0, 10):
    show_random(results[~results.fragment])

geringer mütterlicher und          neonataler Morbidität
geringer mütterlicher und geringer neonataler Morbidität
                          ^^^^^^^^^^^^^^^^^^^           
05_zervixkarzinom_0386.tsv 16 fragment: False
...........

krankheits            - und therapieassoziierte Morbidität
krankheitsassoziierte  und therapieassoziierte Morbidität
^^^^^^^^^^^^^^^^^^^^^                                    
28_komplementaermedizin_0303.tsv 1 fragment: False
...........

Senkung von erektilen Dysfunktionen oder     urologischen Symptomen
Senkung von erektilen Dysfunktionen oder von urologischen Symptomen
                                         ^^^^^^^^^^^^^^^^          
28_komplementaermedizin_0147.tsv 2 fragment: False
...........

erhöhtes Risiko für Fernmetastasen bei Invasion perikolischer Venen [ REF ] , [ REF ] ,     angiolymphatischer Invasion [ REF ] , [ REF ] ,     G3 / G4 Tumoren oder     pT2 - Tumoren
erhöhtes Risiko für Fernmetastasen bei Invasion perikolischer Venen [ REF ] , [ 

In [110]:
for _ in range(0, 10):
    show_random(results[~results.fragment])

Virtuellen Koloskopie ( bei stenosierenden Tumoren und     inkompletter Koloskopie
Virtuellen Koloskopie ( bei stenosierenden Tumoren und bei inkompletter Koloskopie
                                                       ^^^^^^^^^^^^^^^^           
02_kolorektales-karzinom_0197.tsv 1 fragment: False
...........

neuen Behandlungsoptionen der letzten Dekade mit Immun - und     zielgerichteten Therapien
neuen Behandlungsoptionen der letzten Dekade mit Immun - und mit zielgerichteten Therapien
                                                             ^^^^^^^^^^^^^^^^^^^          
07_malignes-melanom_0238.tsv 39 fragment: False
...........

        II
Stadium II
^^^^^^^^^^
08_aktinische-keratose-und-plattenepithelkarzinom-der-haut_0174.tsv 8 fragment: False
...........

adäquater Sicherheitssäume für Organbeweglichkeit und     Lagerungsungenauigkeit sowie     Patientinnenbeweglichkeit
adäquater Sicherheitssäume für Organbeweglichkeit und für Lagerungsungenauigkeit sowie für Patientinnen

In [111]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")

In [112]:
date

'20220512_130813'

In [113]:
results.to_excel(f'ellipses_no_hyphens_{date}.xlsx')